In [ ]:
# Spotify User Churn Prediction
## Step 1: Data Exploration

Objective:
Understand user behavior data and identify patterns related to churn.


In [10]:
import pandas as pd

df = pd.read_csv("../data/spotify_raw.csv")
df.head()


,Age,Gender,spotify_usage_period,spotify_listening_device,spotify_subscription_plan,premium_sub_willingness,preffered_premium_plan,preferred_listening_content,fav_music_genre,music_time_slot,music_Influencial_mood,music_lis_frequency,music_expl_method,music_recc_rating,pod_lis_frequency,fav_pod_genre,preffered_pod_format,pod_host_preference,preffered_pod_duration,pod_variety_satisfaction
0,20-35,Female,More than 2 years,Smart speakers or voice assistants,Free (ad-supported),Yes,Family Plan-Rs 179/month,Podcast,Melody,Night,Sadness or melancholy,leisure time,Playlists,3,Daily,Comedy,Interview,Both,Both,Ok
1,12-20,Male,More than 2 years,Computer or laptop,Free (ad-supported),Yes,Individual Plan- Rs 119/ month,Podcast,Rap,Afternoon,Social gatherings or parties,Workout session,Playlists,2,Several times a week,Comedy,Interview,Both,NaN,Satisfied
2,35-60,Others,6 months to 1 year,Smart speakers or voice assistants,Free (ad-supported),Yes,Student Plan-Rs 59/month,Podcast,Pop,Night,Relaxation and stress relief,"Study Hours, While Traveling",Playlists,4,Once a week,Sports,Interview,NaN,Both,Satisfied
3,20-35,Female,1 year to 2 years,"Smartphone, Smart speakers or voice assistants",Free (ad-supported),No,NaN,Music,Melody,Night,"Relaxation and stress relief, Social gathering...","Office hours, Workout session, leisure time","recommendations, Playlists",4,Never,NaN,NaN,NaN,NaN,Ok
4,20-35,Female,1 year to 2 years,Smartphone,Free (ad-supported),No,NaN,Music,Melody,Night,Relaxation and stress relief,leisure time,"recommendations, Playlists",4,Rarely,Lifestyle and Health,Story telling,Well known individuals,Both,Ok


In [11]:
df.shape

(520, 20)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Age                          520 non-null    object
 1   Gender                       520 non-null    object
 2   spotify_usage_period         520 non-null    object
 3   spotify_listening_device     520 non-null    object
 4   spotify_subscription_plan    520 non-null    object
 5   premium_sub_willingness      520 non-null    object
 6   preffered_premium_plan       312 non-null    object
 7   preferred_listening_content  520 non-null    object
 8   fav_music_genre              520 non-null    object
 9   music_time_slot              520 non-null    object
 10  music_Influencial_mood       520 non-null    object
 11  music_lis_frequency          520 non-null    object
 12  music_expl_method            520 non-null    object
 13  music_recc_rating            520 no

In [13]:
df.isnull().sum()

Age                              0
Gender                           0
spotify_usage_period             0
spotify_listening_device         0
spotify_subscription_plan        0
premium_sub_willingness          0
preffered_premium_plan         208
preferred_listening_content      0
fav_music_genre                  0
music_time_slot                  0
music_Influencial_mood           0
music_lis_frequency              0
music_expl_method                0
music_recc_rating                0
pod_lis_frequency                0
fav_pod_genre                  148
preffered_pod_format           140
pod_host_preference            141
preffered_pod_duration         129
pod_variety_satisfaction         0
dtype: int64

In [16]:
df.columns


Index(['Age', 'Gender', 'spotify_usage_period', 'spotify_listening_device',
       'spotify_subscription_plan', 'premium_sub_willingness',
       'preffered_premium_plan', 'preferred_listening_content',
       'fav_music_genre', 'music_time_slot', 'music_Influencial_mood',
       'music_lis_frequency', 'music_expl_method', 'music_recc_rating',
       'pod_lis_frequency', 'fav_pod_genre', 'preffered_pod_format',
       'pod_host_preference', 'preffered_pod_duration',
       'pod_variety_satisfaction'],
      dtype='object')

CREATE CHURN LABEL

In [17]:
df[['music_lis_frequency', 'premium_sub_willingness', 'music_recc_rating']].head()


,music_lis_frequency,premium_sub_willingness,music_recc_rating
0,leisure time,Yes,3
1,Workout session,Yes,2
2,"Study Hours, While Traveling",Yes,4
3,"Office hours, Workout session, leisure time",No,4
4,leisure time,No,4


In [18]:
df['music_lis_frequency'].value_counts()
df['premium_sub_willingness'].value_counts()
df['music_recc_rating'].value_counts()


music_recc_rating
3    190
4    174
5     86
2     56
1     14
Name: count, dtype: int64

In [19]:
df['churn'] = 0

df.loc[
    (df['music_lis_frequency'].isin(['Rarely', 'Occasionally'])) &
    (df['premium_sub_willingness'] == 'No') &
    (df['music_recc_rating'] <= 2),
    'churn'
] = 1


In [20]:
df['churn'].value_counts()


churn
0    520
Name: count, dtype: int64

In [21]:
df['churn'] = 0


In [22]:
df.loc[
    (
        (df['music_lis_frequency'].isin(['Rarely', 'Occasionally'])) &
        (df['premium_sub_willingness'] == 'No')
    )
    |
    (
        (df['music_lis_frequency'].isin(['Rarely', 'Occasionally'])) &
        (df['music_recc_rating'] <= 2)
    )
    |
    (
        (df['premium_sub_willingness'] == 'No') &
        (df['music_recc_rating'] <= 2)
    ),
    'churn'
] = 1


In [23]:
df['churn'].value_counts()


churn
0    481
1     39
Name: count, dtype: int64

Dataset is imbalanced (~7.5% churn)

Observation:
- The dataset contains user listening behavior metrics.
- Churn is the target variable (1 = churned, 0 = active).


In [24]:
df = df.dropna()


In [25]:
df.to_csv("../data/spotify_processed.csv", index=False)
